In [3]:
import pandas as pd
import marvin
import re
from marvin.tools import Cube , Image
from marvin import config
from astropy.io import ascii
from marvin.tools.quantities.map import Map
import numpy as np
import matplotlib.pyplot as plt
from zmq import EVENT_CLOSE_FAILED
from marvin.tools import Maps 
import matplotlib.style as style 
from matplotlib.pyplot import cm
from scipy import sparse, interpolate
from matplotlib import colors
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from marvin.tools.maps import Maps
import marvin.utils.plot.map as mapplot
from marvin.utils.general.images import showImage
from matplotlib.colors import ListedColormap
from matplotlib.pyplot import cm

config.access = 'collab'
config.login()
config.setRelease('MPL-11')

class WHAN:
    
    """MODULE WHAN, this creates a map of ionization for activity nuclear on the spaxel of galaxy"""
    
    def __init__(self,plateifu,snr=1,**kwargs):
        
        self.__dict__.update(kwargs)
        
        
        self.plateifu = plateifu
        self.maps = Maps(plateifu=self.plateifu)
        
        self.niiha = self.maps.emline_gflux_nii_6585/self.maps.emline_gflux_ha_6564
        self.ewha = self.maps['emline_sew_ha_6564']
        self.ewnii = self.maps['emline_sew_nii_6585']
        
        self.logniiha = np.log10(self.niiha.value)
        
        self.niiha_low_snr = mapplot.mask_low_snr(self.niiha.value, self.niiha.ivar, snr_min=snr)
        self.ewha_low_snr = mapplot.mask_low_snr(self.ewha.value, self.ewha.ivar, snr_min=snr)
        self.ewnii_low_snr = mapplot.mask_low_snr(self.ewnii.value, self.ewnii.ivar, snr_min=snr)
        self.low_snr = np.logical_or(self.niiha_low_snr,self.ewha_low_snr, self.ewnii_low_snr)
        
        
        self.ivar = self.ewha.value.copy()
        self.ivar[self.low_snr] = 0
        self.nocov = self.ewha.pixmask.get_mask('NOCOV')
        
    def diagram_whan(self,path = None, **kwargs):
        
        mask = self.nocov | self.low_snr
        m_logniiha = self.logniiha[mask == 0]
        m_ewha = self.ewha.value[mask == 0]
        m_ewnii = self.ewnii.value[mask == 0]

        m_psf1 = (m_logniiha < -0.4) & (m_logniiha > -0.46) & (m_ewha > 3)
        m_psf2 = (m_logniiha < -0.46) & (m_logniiha > -0.52) & (m_ewha > 3)
        m_psf3 = (m_logniiha < -0.52) & (m_logniiha > -0.58) & (m_ewha > 3)
        m_psf4 = (m_logniiha < -0.58) & (m_ewha > 3)

        m_sagn1 = (m_logniiha > -0.4) & (m_logniiha < -0.35) & (m_ewha > 6)
        m_sagn2 = (m_logniiha > -0.35) & (m_logniiha < -0.3) & (m_ewha > 6)
        m_sagn3 = (m_logniiha > -0.3) & (m_logniiha < -0.25) & (m_ewha > 6)
        m_sagn4 = (m_logniiha > -0.25) & (m_ewha > 6)

        m_wagn1 = (m_logniiha> -0.4) & ((m_ewha > 5.25) & (m_ewha < 6))
        m_wagn2 = (m_logniiha> -0.4) & ((m_ewha > 4.5) & (m_ewha < 5.25))
        m_wagn3 = (m_logniiha> -0.4) & ((m_ewha > 3.75) & (m_ewha < 4.5))
        m_wagn4 = (m_logniiha> -0.4) & ((m_ewha > 3) & (m_ewha < 3.75))

        m_rg1 = (m_ewha < 3) & (m_ewha > 2.25) 
        m_rg2 = (m_ewha < 2.25) & (m_ewha > 1.5) 
        m_rg3 = (m_ewha < 1.5) & (m_ewha > 0.75)
        m_rg4 = (m_ewha < 0.75)

        m_pg = (m_ewha < 0.5) & (m_ewnii < 0.5)

        WHAN_color = ['red', 'blue', 'goldenrod', 'green', 'purple']
        
        plt.figure()
        plt.plot(self.logniiha[mask == 0], self.ewha.value[mask == 0], 'o', alpha=0.7)

        plt.plot(m_logniiha[m_psf1], m_ewha[m_psf1], 'o', color='lightblue', alpha=0.7)
        plt.plot(m_logniiha[m_psf2], m_ewha[m_psf2], 'o', color='lightsteelblue', alpha=0.7)
        plt.plot(m_logniiha[m_psf3], m_ewha[m_psf3], 'o', color='cornflowerblue', alpha=0.7)
        plt.plot(m_logniiha[m_psf4], m_ewha[m_psf4], 'o', color='royalblue', alpha=0.7, label=r'$\rm PSF$')

        plt.plot(m_logniiha[m_sagn1], m_ewha[m_sagn1], 'o', color='plum', alpha=0.7)
        plt.plot(m_logniiha[m_sagn2], m_ewha[m_sagn2], 'o', color='mediumorchid', alpha=0.7)
        plt.plot(m_logniiha[m_sagn3], m_ewha[m_sagn3], 'o', color='darkorchid', alpha=0.7)
        plt.plot(m_logniiha[m_sagn4], m_ewha[m_sagn4], 'o', color='darkviolet', alpha=0.7, label=r'$\rm sAGN$')

        plt.plot(m_logniiha[m_wagn1], m_ewha[m_wagn1], 'o',color='seagreen', alpha=0.7)
        plt.plot(m_logniiha[m_wagn2], m_ewha[m_wagn2], 'o',color='darkseagreen', alpha=0.7)
        plt.plot(m_logniiha[m_wagn3], m_ewha[m_wagn3], 'o',color='lightgreen', alpha=0.7, label=r'$\rm wAGN$')
        plt.plot(m_logniiha[m_wagn4], m_ewha[m_wagn4], 'o',color='palegreen', alpha=0.7)

        plt.plot(m_logniiha[m_rg1], m_ewha[m_rg1], 'o', color='peachpuff' , alpha=0.7)
        plt.plot(m_logniiha[m_rg2], m_ewha[m_rg2], 'o', color='sandybrown' , alpha=0.7, label=r'$\rm RG$')
        plt.plot(m_logniiha[m_rg3], m_ewha[m_rg3], 'o', color='coral' , alpha=0.7)
        plt.plot(m_logniiha[m_rg4], m_ewha[m_rg4], 'o', color='tomato' , alpha=0.7)

        plt.plot(m_logniiha[m_pg], m_ewha[m_pg], 'o', color='red' , alpha=0.7, label=r'$\rm Passive$')

        plt.yscale('log')
        plt.title(r'$\rm WHAN~Diagram~ {}$'.format(self.plateifu), fontsize=16)
        plt.xlabel(r'$\rm log~[N_{II}]/H_\alpha$', fontsize=16)
        plt.ylabel(r'$\rm W_{H_\alpha}~[\AA]$', fontsize=16)
        plt.legend()
        plt.show()
        if path == None:
            pass
        else:
            plt.savefig(path+'Whd_'+self.plateifu+'.png', bbox_inches='tight', pad_inches = 0,transparent=True)

        
        plt.close(**kwargs)  
        
    def map_whan(self,path = None, **kwargs):

        psf1 = (self.logniiha < -0.4) & (self.logniiha > -0.46) & (self.ewha.value > 3)
        psf2 = (self.logniiha < -0.46) & (self.logniiha > -0.52) & (self.ewha.value > 3)
        psf3 = (self.logniiha < -0.52) & (self.logniiha > -0.58) & (self.ewha.value > 3)
        psf4 = (self.logniiha < -0.58) & (self.ewha.value > 3)

        sagn1 = (self.logniiha > -0.4) & (self.logniiha < -0.35) & (self.ewha.value > 6)
        sagn2 = (self.logniiha > -0.35) & (self.logniiha < -0.3) & (self.ewha.value > 6)
        sagn3 = (self.logniiha > -0.3) & (self.logniiha < -0.25) & (self.ewha.value > 6)
        sagn4 = (self.logniiha > -0.25) & (self.ewha.value > 6)

        wagn1 = (self.logniiha> -0.4) & ((self.ewha.value > 5.25) & (self.ewha.value < 6))
        wagn2 = (self.logniiha> -0.4) & ((self.ewha.value > 4.5) & (self.ewha.value < 5.25))
        wagn3 = (self.logniiha> -0.4) & ((self.ewha.value > 3.75) & (self.ewha.value < 4.5))
        wagn4 = (self.logniiha> -0.4) & ((self.ewha.value > 3) & (self.ewha.value < 3.75))

        rg1 = (self.ewha.value < 3) & (self.ewha.value > 2.25) 
        rg2 = (self.ewha.value < 2.25) & (self.ewha.value > 1.5) 
        rg3 = (self.ewha.value < 1.5) & (self.ewha.value > 0.75)
        rg4 = (self.ewha.value < 0.75)

        pg = (self.ewha.value < 0.5) & (self.ewnii.value < 0.5)

        value = self.ewha.value.copy()
        value[psf1] = 1.     
        value[psf2] = 2.
        value[psf3] = 3.
        value[psf4] = 4.

        value[sagn1] = 5.  
        value[sagn2] = 6.  
        value[sagn3] = 7.  
        value[sagn4] = 8.  

        value[wagn1] = 9.   
        value[wagn2] = 10.
        value[wagn3] = 11.
        value[wagn4] = 12.

        value[rg1] = 13.     
        value[rg2] = 14.
        value[rg3] = 15.
        value[rg4] = 16.

        value[pg] = 17.      
        

        cmap = colors.ListedColormap(['lightblue', 'lightsteelblue','cornflowerblue','royalblue',
                                      'plum','mediumorchid','darkorchid','darkviolet',
                                      'seagreen','darkseagreen','lightgreen','palegreen',
                                      'peachpuff','sandybrown','coral','tomato','red'])

        fig, ax, cb = mapplot.plot(value=value, ivar=self.ivar, mask=self.nocov, cmap=cmap, use_masks='NOCOV', return_cb=True, cbrange=(17,1),
                                   title='WHAN MAP {}'.format(self.plateifu))

        cb.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
        cb.set_ticklabels(['PSF','','','','SAGN','','','','WAGN','','','','RG','','','','PG'])
        
        if path == None:
            pass
        else:
            plt.savefig(path+'Wm_'+self.plateifu+'.png', bbox_inches='tight', pad_inches = 0,transparent=True)
        
        plt.close(**kwargs)  
        
        
    def stat_whan(self,**kwargs):
        
        mask = self.nocov | self.low_snr
        m_logniiha = self.logniiha[mask == 0]
        m_ewha = self.ewha.value[mask == 0]
        m_ewnii = self.ewnii.value[mask == 0]

        m_psf1 = (m_logniiha < -0.4) & (m_logniiha > -0.46) & (m_ewha > 3)
        m_psf2 = (m_logniiha < -0.46) & (m_logniiha > -0.52) & (m_ewha > 3)
        m_psf3 = (m_logniiha < -0.52) & (m_logniiha > -0.58) & (m_ewha > 3)
        m_psf4 = (m_logniiha < -0.58) & (m_ewha > 3)

        m_sagn1 = (m_logniiha > -0.4) & (m_logniiha < -0.35) & (m_ewha > 6)
        m_sagn2 = (m_logniiha > -0.35) & (m_logniiha < -0.3) & (m_ewha > 6)
        m_sagn3 = (m_logniiha > -0.3) & (m_logniiha < -0.25) & (m_ewha > 6)
        m_sagn4 = (m_logniiha > -0.25) & (m_ewha > 6)

        m_wagn1 = (m_logniiha> -0.4) & ((m_ewha > 5.25) & (m_ewha < 6))
        m_wagn2 = (m_logniiha> -0.4) & ((m_ewha > 4.5) & (m_ewha < 5.25))
        m_wagn3 = (m_logniiha> -0.4) & ((m_ewha > 3.75) & (m_ewha < 4.5))
        m_wagn4 = (m_logniiha> -0.4) & ((m_ewha > 3) & (m_ewha < 3.75))

        m_rg1 = (m_ewha < 3) & (m_ewha > 2.25) 
        m_rg2 = (m_ewha < 2.25) & (m_ewha > 1.5) 
        m_rg3 = (m_ewha < 1.5) & (m_ewha > 0.75)
        m_rg4 = (m_ewha < 0.75)

        m_pg = (m_ewha < 0.5) & (m_ewnii < 0.5)

        
        stat_ =mask
        
        psf_stat = len(m_psf1[m_psf1==True])+len(m_psf2[m_psf2==True])+len(m_psf3[m_psf3==True])+len(m_psf4[m_psf4==True])
        sagn_stat = len(m_sagn1[m_sagn1==True])+len(m_sagn2[m_sagn2==True])+len(m_sagn3[m_sagn3==True])+len(m_sagn4[m_sagn4==True])

        wagn_stat = len(m_wagn1[m_wagn1==True])+len(m_wagn2[m_wagn2==True])+len(m_wagn3[m_wagn3==True])+len(m_wagn4[m_wagn4==True])
        rg_stat = len(m_rg1[m_rg1==True])+len(m_rg2[m_rg2==True])+len(m_rg3[m_rg3==True])+len(m_rg4[m_rg4==True])

        pg_stat = len(m_pg[m_pg==True])
        all_stat = psf_stat+sagn_stat+wagn_stat+rg_stat+pg_stat
        
        return psf_stat,sagn_stat,wagn_stat,rg_stat,pg_stat, all_stat

[WARNING]: path /home/paulo/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
